In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# MODEL FROM THE RESEARCH PAPER

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # kernel
        # (32-5/1)+1 = ouput of each filter
        # output all filters = 28 x92 = 2576 
        self.conv1 = nn.Conv3d(1,96,(5,5,5), padding=0)
        self.pool  = nn.MaxPool3d(2, stride=2)
        # you can use this formula [(W−K+2P)/S]+1.
        self.conv2  =  nn.Conv3d(96,192,(3,3,3), padding=0)
        self.pool1  =  nn.MaxPool3d(2, stride=2)
        self.conv3  =  nn.Conv3d(192,384,(3,3,3), padding=0)
        self.pool2  =  nn.MaxPool3d(2, stride=2)
        
        self.ln1    =  nn.Linear(in_features=3072, out_features=4096)
        self.ln2    =  nn.Linear(in_features=4096, out_features=1024)
        self.out1    =  nn.Linear(in_features=1024, out_features=30)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.pool2(x)
        x = x.view(-1, 3072)

        x = self.ln1(x)
        x = F.relu(x)


       
        x = self.ln2(x)
        x = F.relu(x)


        x1 = self.out1(x)
        ret = x1.view(-1, 30)
         
        return ret

    def num_flat_features(self, x):
        x = self.x
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features    
    

    
    
net = Net()
print(net)

net.to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# MODIFIED MODEL FOR OUR APPROACH
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(1,48,(5,5,5), padding=1)
        self.conv1_1 = nn.Conv3d(1,48,(3,3,3), padding=0)
        
        self.pool   =  nn.MaxPool3d(2, stride=2)
        
        self.conv2    =    nn.Conv3d(96,96,(5,5,5), padding=1)
        self.conv2_2  =  nn.Conv3d(96,96,(3,3,3), padding=0)        
        
        self.pool1  =  nn.MaxPool3d(2, stride=2)
        
        self.conv3  =  nn.Conv3d(192,192,(5,5,5), padding=1)
        self.conv3_1  =  nn.Conv3d(192,192,(3,3,3), padding=0)
        
        self.pool2  =  nn.MaxPool3d(2, stride=2)        
        
        
        self.ln1    =  nn.Linear(in_features=3072, out_features=4096)
        self.ln2    =  nn.Linear(in_features=4096, out_features=1024)
        self.out1   = nn.Linear(in_features=1024, out_features=30)

        
    
    def forward(self, x):
        #branch one
        x_1 = self.conv1(x)
        x_2 = self.conv1_1(x)
        

        x_3 = torch.cat([x_2, x_1], dim=1)
        

        x_4_4 = self.pool(x_3)
        
        x_4_5 = self.conv2(x_4_4)
        x_4_1 = self.conv2_2(x_4_4)
        
        x_4 = torch.cat([x_4_1, x_4_5], dim=1)
        
        
        
        x_4 = F.relu(x_4)
        x_4 = self.pool1(x_4)
        
        
        x_4_6 = self.conv3(x_4)
        x_4_7 = self.conv3_1(x_4)
        
        x_4 = torch.cat([x_4_6, x_4_7], dim=1)
        
        x_4 = F.relu(x_4)
        
        
        x_4 = self.pool2(x_4)
        x_4 = x_4.view(-1, 3072)
    
        
        
        x_4 = self.ln1(x_4)
        x_4 = F.relu(x_4)      
        x_4 = self.ln2(x_4)
        x_4 = F.relu(x_4)
        x_4 = self.out1(x_4)
        ret = x_4.view(-1, 30)
         
        return ret

def num_flat_features(x):
    x = x
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features    
    

    
    
net = Net()
print(net)

net.to(device)

Net(
  (conv1): Conv3d(1, 48, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv1_1): Conv3d(1, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(96, 96, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2_2): Conv3d(96, 96, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv3d(192, 192, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3_1): Conv3d(192, 192, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (ln1): Linear(in_features=3072, out_features=4096, bias=True)
  (ln2): Linear(in_features=4096, out_features=1024, bias=True)
  (out1): Linear(in_features=1024, out_features=30, bias=True)
)


Net(
  (conv1): Conv3d(1, 48, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv1_1): Conv3d(1, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(96, 96, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2_2): Conv3d(96, 96, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv3d(192, 192, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3_1): Conv3d(192, 192, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (ln1): Linear(in_features=3072, out_features=4096, bias=True)
  (ln2): Linear(in_features=4096, out_features=1024, bias=True)
  (out1): Linear(in_features=1024, out_features=30, bias=True)
)

In [ ]:
class Generator():
    def __init__(self):
      #  pass
        import numpy as np
        import pandas as pd
        import h5py
        self.NYU_X = pd.read_csv('path to 30 point ground truth')
 

    def get_data(self,file_no):
        import h5py
        import numpy as np                     

        filename = '/path to tsdf link given in repo /TSDF/'+str(file_no)+'.h5'
        h5 = h5py.File(filename,'r')
        input = np.array(h5['TSDF'])
        input = np.reshape(input,(1,1,32,32,32))
                # VSTOXX futures data
        h5.close()           
        inputs = np.array(input).tolist()
        inputs = torch.FloatTensor(inputs)

        output1 = self.NYU_X.iloc[file_no].values

                                                                       

        output1 = output1[0:30]


        output1 = np.asarray(output1)

        output  = torch.from_numpy(output1).float()
        output  = torch.reshape(output, (1, 30))

        return inputs,output

In [ ]:
gen = Generator()
net = Net()
import pandas as pd
import torch
import numpy as np
import numpy 

inputs , labels = gen.get_data(1126)
#inputs = np.random.rand(1,1,32,32,32)
print(labels)
print(np.shape(labels))
#inputs = inputs.detach().numpy()
#inputs = torch.from_numpy(inputs).float()

outputs = net(inputs)
print(np.shape(outputs))
print(outputs)

tensor([[-0.5553,  0.2882, -0.3079, -0.1309, -0.0493,  0.1058, -0.1873,  0.3007,
         -0.0166, -0.1716, -0.0343, -0.0783,  0.2015, -0.1207, -0.0424, -0.0295,
          0.0045,  0.3447,  0.0474, -0.0570, -0.1766,  0.1464, -0.0423, -0.0785,
          0.0340,  0.0046,  0.0244, -0.0105, -0.0030, -0.0048]])
torch.Size([1, 30])
torch.Size([1, 30])
tensor([[-0.0067, -0.0077,  0.0029, -0.0223, -0.0313,  0.0088, -0.0120,  0.0217,
         -0.0163,  0.0254,  0.0025,  0.0208, -0.0614, -0.0257,  0.0048,  0.0187,
          0.0057,  0.0218,  0.0376,  0.0066,  0.0044, -0.0346, -0.0143, -0.0230,
         -0.0004, -0.0134, -0.0140, -0.0096, -0.0188,  0.0126]],
       grad_fn=<ViewBackward>)


In [ ]:
criterion = nn.L1Loss()

In [ ]:
net = Net()
net.to(device)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torch.optim as optim
import numpy as np
import numpy
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.Adam(net.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0005, amsgrad=False)

gen = Generator()
import random

for epoch in range(1,16):  # loop over the dataset multiple times
    running_loss = 0.0
    i = 0
    if epoch  == 5:
      for g in optimizer.param_groups:
           g['lr'] = 0.001
    if epoch == 10:
       for g in optimizer.param_groups:
           g['lr'] = 0.0001


    for i in range(0,3500):
        # get the inputs; data is a list of [inputs, labels]
        k=0
        file_no = random.randint(0, 60000)
        for k in range (0,16):
            file_no = file_no + 1
            if k == 0 :
                  inputs, labels = gen.get_data(file_no)
            if k > 0 :      
                  inputs_stack, labels_stack = gen.get_data(file_no)
                  inputs = torch.cat([inputs, inputs_stack])
                  labels = torch.cat([labels, labels_stack])


        
        INN = inputs.to(device)
        OUT = labels.to(device)
        optimizer.zero_grad()
        outputs = net(INN)
        loss = criterion(outputs, OUT)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        LOSS = 0.0
        if i == 3499:    # print every 2000 mini-batches# CHANGE THIS VALUE
            print('[%d, %5d] loss: %.5f' %
                  (epoch , i + 1, running_loss / 3500))
            running_loss = 0.0
            
            g = 0
            for g in range(0,750):
                        k=0
                        file_no = random.randint(60012, 72730)
                        for k in range (0,16):
                            file_no = file_no + 1
                            if k == 0 :
                                    inputs, labels = gen.get_data(file_no)
                            if k > 0 :      
                                    inputs_stack, labels_stack = gen.get_data(file_no)
                                    inputs = torch.cat([inputs, inputs_stack])
                                    labels = torch.cat([labels, labels_stack])
 
                  

                 

                 

                        INN = inputs.to(device)
                        OUT = labels.to(device)
           # zero the parameter gradients
                        optimizer.zero_grad()
        # forward + backward + optimize
                        outputs = net(INN)
                        loss = criterion(outputs, OUT)
       #    loss = criterion(outputs, labels)
                        LOSS  += loss.item()
            LOSS = LOSS/750
            print('Avergae_validation_loss was='+str(LOSS) )
            running_loss = 0.0
            if epoch ==1 :
                best_loss = LOSS
                torch.save(net, '/content/drive/My Drive/FYP data/models/model1/'+str('pca_30_points_relational_model_2.pt'))
            if LOSS < best_loss:
                best_loss = LOSS
                torch.save(net, '/content/drive/My Drive/FYP data/models/model1/'+str('pca_30_points_relational_model_2.pt'))
    
            LOSS = 0    
            running_loss = 0.0   

print('Finished Training')

[1,  3500] loss: 0.11389
Avergae_validation_loss was=0.1045650735149781
[2,  3500] loss: 0.07828
Avergae_validation_loss was=0.08975564017146825
[3,  3500] loss: 0.06349
Avergae_validation_loss was=0.08068001163502535
[4,  3500] loss: 0.05489
Avergae_validation_loss was=0.07816365649054448
[5,  3500] loss: 0.04264
Avergae_validation_loss was=0.06821760315572221
[6,  3500] loss: 0.03812
Avergae_validation_loss was=0.06963699182619651
[7,  3500] loss: 0.03592
Avergae_validation_loss was=0.06884313842530052
[8,  3500] loss: 0.03451
Avergae_validation_loss was=0.06673060000191132
[9,  3500] loss: 0.03363
Avergae_validation_loss was=0.0684969586705168
[10,  3500] loss: 0.03189
Avergae_validation_loss was=0.06385468562444051
[11,  3500] loss: 0.03139
Avergae_validation_loss was=0.06693707218766212
[12,  3500] loss: 0.03136
Avergae_validation_loss was=0.06517734333872795
[13,  3500] loss: 0.03153
Avergae_validation_loss was=0.0663954989425838
[14,  3500] loss: 0.03101
Avergae_validation_loss 

In [ ]:
# EXTRACTING THE TSDF FILE IN ZIP /// DOWNLOAD FROM LINK PLZ
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/FYP data/nyu dataset/TSDF.zip', 'r') as zip_ref:
    zip_ref.extractall()